<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#VGG-Blocks" data-toc-modified-id="VGG-Blocks-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>VGG Blocks</a></span></li></ul></div>

In [1]:
import mxnet
from mxnet import gluon,npx,np,autograd
from mxnet.gluon import nn
npx.set_np()

## VGG Blocks
<img src='../images/vvg.jpg'>
The function takes two arguments corresponding to the number of convolutional layers num_convs and the number of output channels num_channels

In [2]:
def vgg_block(num_convs, num_channels):
    blk = nn.Sequential()
    for _ in range(num_convs):
        blk.add(nn.Conv2D(num_channels, kernel_size=3,padding=1, activation='relu'))
    blk.add(nn.MaxPool2D(pool_size=2, strides=2))
    return blk

The original VGG network had 5 convolutional blocks, among which the first two have one convolutional layer each and the latter three contain two convolutional layers each. The first block has
64 output channels and each subsequent block doubles the number of output channels, until that
number reaches 512. Since this network uses 8 convolutional layers and 3 fully-connected layers,
it is often called VGG-11.

In [3]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

The following code implements VGG-11. This is a simple matter of executing a for loop over
conv_arch.

In [5]:
def vgg(conv_arch):
    net=nn.Sequential()
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs,num_channels))
    net.add(nn.Dense(4096, activation='relu'), nn.Dropout(0.5),nn.Dense(4096, activation='relu'),
            nn.Dropout(0.5),nn.Dense(10))
    return net

Next, we will construct a single-channel data example with a height and width of 224 to observe
the output shape of each layer.

In [6]:
vgg11 = vgg(conv_arch)
vgg11.initialize()

In [7]:

X = np.random.uniform(size=(1, 1, 224, 224))
for blk in vgg11:
    X = blk(X)
    print(blk.name, 'output shape:\t', X.shape)

sequential1 output shape:	 (1, 64, 112, 112)
sequential2 output shape:	 (1, 128, 56, 56)
sequential3 output shape:	 (1, 256, 28, 28)
sequential4 output shape:	 (1, 512, 14, 14)
sequential5 output shape:	 (1, 512, 7, 7)
dense0 output shape:	 (1, 4096)
dropout0 output shape:	 (1, 4096)
dense1 output shape:	 (1, 4096)
dropout1 output shape:	 (1, 4096)
dense2 output shape:	 (1, 10)
